In [ ]:
import gosling as gos
import tracks

clin_var_beddb = gos.beddb(
    "../data/agg/clinvar.bed.beddb",
    genomicFields=[{"index": 1, "name": "start"}, {"index": 2, "name": "end"}],
    valueFields=[{"index": 7, "name": "significance", "type": "nominal"}],
)

categories = [
    "Benign", "Benign/Likely_benign", "Likely_benign", 
    "Uncertain_significance", "Likely_pathogenic", 
    "Pathogenic/Likely_pathogenic", "Pathogenic",
]

colors = ["#5A9F8C", "#5A9F8C", "#029F73", "gray", "#CB96B3", "#CB71A3", "#CB3B8C"]

clin_var_multivec = gos.multivec(
    "../data/agg/density.multires.mv5",
    row="significance",
    column="position",
    value="count",
    categories=categories,
    binSize=4,
)

strips = gos.Track(clin_var_beddb).mark_bar().encode(
    x="start:G",
    y=gos.Y("significance:N", domain=categories, baseline="Uncertain_significance"),
    size=gos.value(1),
    color=gos.value("lightgray"),
    stroke=gos.value("lightgray"),
    strokeWidth=gos.value(1),
    opacity=gos.value(0.3),
).visibility_lt(
    measure="zoomLevel",
    target="mark",
    threshold=100000,
    transitionPadding=100000,
)

points = gos.Track(clin_var_beddb).mark_point().encode(
    x="start:G",
    color=gos.Color("significance:N", domain=categories, range=colors),
    row=gos.Row("significance:N", domain=categories),
    size=gos.value(7),
    opacity=gos.value(0.8),
).visibility_lt(
    measure="zoomLevel",
    target="mark",
    threshold=1000000,
    transitionPadding=1000000,
)

bars = gos.Track(clin_var_multivec).mark_bar().encode(
    x="start:G",
    xe="end:G",
    y=gos.Y("count:Q", axis="none"),
    color=gos.Color("significance:N", domain=categories, range=colors, legend=True)
).visibility_gt(
    measure="zoomLevel",
    target="mark",
    threshold=500000,
    transitionPadding=500000,
)

lolipop = gos.overlay(strips, points, bars).properties(
    width=725,
    height=150,
    xDomain=gos.GenomicDomain(chromosome="13", interval=[31500000, 33150000]),
)

# TODO: Need to specify top-level 
view = gos.stack(
    tracks.test.properties(id="view1", height=95, width=725),
    lolipop.properties(height=150, width=150),
).properties(
    id="view1",
    xDomain=gos.GenomicDomain(chromosome="13", interval=[31500000, 33150000]),
)

view

In [ ]:
view